# Toy problem for TLS solution
We want to solve here the identification problem with TLS in a very simple case (a network with 2 nodes, without any structural contraint.

In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp

In [ ]:
import sys
sys.path.insert(1, '..')

from src.models.error_in_variable import TotalLeastSquares, SparseTotalLeastSquare
from src.models.regression import ComplexLasso
from src.identification.error_metrics import fro_error, error_metrics

In [ ]:
np.set_printoptions(precision=2)

In [ ]:
samples = 300
v_noise_sd = 0.01
i_noise_sd = 0.01

In [ ]:
np.random.seed(11)
y_bus = np.array([
    [1+1j, 0, 0, 0],
    [0, 2+1j, 0, 0],
    [2+1j, 0, 1+1j, 0],
    [0, 0, 0, 2+1j],
])
nodes = y_bus.shape[0]
real_voltages = np.random.normal(1, 0.1, (samples, nodes)) + 1j*np.random.normal(1, 0.1, (samples, nodes))
real_currents = real_voltages @ y_bus

voltages = real_voltages.copy() + np.random.normal(0, v_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, v_noise_sd, (samples, nodes))
currents = real_currents.copy() + np.random.normal(0, i_noise_sd, (samples, nodes)) + 1j*np.random.normal(0, i_noise_sd, (samples, nodes))

voltage_error = voltages - real_voltages
current_error = currents - real_currents

# Standard TLS

In [ ]:
eiv = TotalLeastSquares()
eiv.fit(voltages, currents)
y_eiv = eiv.fitted_admittance_matrix
y_eiv

In [ ]:
error_metrics(y_bus, y_eiv)

# Standard LASSO

In [ ]:
lasso = ComplexLasso(y_bus, verbose=False, lambdas=np.logspace(-4, 0, 20))
lasso.fit(voltages, currents)
y_lasso = lasso.fitted_admittance_matrix
y_lasso

In [ ]:
error_metrics(y_bus, y_lasso)

In [ ]:
lasso.best_trial

# L1-regularized TLS

In [ ]:
tls_lambda = lasso.best_trial.hyperparameters['lambda']
sparse_eiv = SparseTotalLeastSquare(lambda_value=tls_lambda, abs_tol=10e-12, rel_tol=10e-12, solver=cp.GUROBI, max_iterations=50)
sparse_eiv.fit(voltages, currents)
y_sparse_eiv = sparse_eiv.fitted_admittance_matrix
y_sparse_eiv

In [ ]:
error_metrics(y_bus, y_sparse_eiv)

In [ ]:
y_errors = pd.Series([fro_error(y_bus, i.fitted_parameters) for i in sparse_eiv.iterations])
pd.Series(y_errors).plot(title='Fro error on Y')

In [ ]:
targets = pd.Series([i.target_function for i in sparse_eiv.iterations])
pd.Series(targets).plot(title='Target function')

In [ ]:
np.abs(y_bus - y_sparse_eiv)

In [ ]:
np.abs(y_bus - y_eiv)

In [ ]:
np.abs(y_bus - y_sparse_eiv) > np.abs(y_bus - y_eiv)